# Расчёт бизнес-метрик

In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option("display.precision", 2)

In [2]:
# устанавливаем параметры
db_config = {'user': '', # имя пользователя
             'pwd': '',  # пароль
             'host': '', # хост
             'port': '', # порт подключения
             'db': ''}   # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db'])

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string)

In [5]:
def sql_request(query):
    """
    Функция запроса к базе данных и возврата ответа в виде pandas датафрейма

    query - запрос
    """

    return pd.read_sql(query, con = engine)

## Расчёт Lifetime Value

In [6]:
query = """
SELECT DATE_TRUNC('month', u.created_at)::date,
       SUM(o.total_amt) / COUNT(DISTINCT o.user_id) AS ltv
FROM tools_shop.orders AS o
JOIN tools_shop.users AS u ON o.user_id = u.user_id
GROUP BY DATE_TRUNC('month', u.created_at)::date;
"""

In [7]:
sql_request(query)

,date_trunc,ltv
0,2016-02-01,9.42
1,2016-03-01,251.38
2,2016-04-01,265.82
3,2016-05-01,231.30
4,2016-06-01,231.67
...,...,...
59,2021-01-01,267.78
60,2021-02-01,255.01
61,2021-03-01,279.13
62,2021-04-01,314.17


## Расчёт конверсии

In [8]:
# Общая конверсию из пользователя в клиента
query = """
SELECT ROUND(COUNT(DISTINCT o.user_id) * 100.0 / COUNT(DISTINCT u.user_id), 1)
FROM tools_shop.orders AS o
RIGHT JOIN tools_shop.users AS u ON o.user_id = u.user_id
"""

In [9]:
sql_request(query)

,round
0,14.9


## Расчёт ARPPU

In [10]:
# ARPPU в разрезе года оформления заказа
query = """
SELECT DATE_TRUNC('year', o.created_at)::date AS year,
       ROUND(SUM(o.total_amt) / COUNT(DISTINCT u.user_id), 2) AS arppu
FROM tools_shop.users u
JOIN tools_shop.orders o ON o.user_id = u.user_id
GROUP BY year;
"""

In [11]:
sql_request(query)

,year,arppu
0,2016-01-01,264.61
1,2017-01-01,266.38
2,2018-01-01,270.71
3,2019-01-01,265.21
4,2020-01-01,282.91
5,2021-01-01,266.95


## Расчёт ROI. Динамика возврата инвестиций

In [12]:
# ROI в динамике по месяцам
query = """
WITH a AS (
    SELECT CAST(DATE_TRUNC('month', created_at) AS DATE) AS dt,
           SUM(costs) AS costs
    FROM tools_shop.costs
    GROUP BY dt
    ),
    b AS (
    SELECT CAST(DATE_TRUNC('month', created_at) AS DATE) AS dt, 
           SUM(total_amt) AS revenue
    FROM tools_shop.orders
    GROUP BY dt   
    )
SELECT a.dt,
       b.revenue * 100 / a.costs AS ROI
FROM a
JOIN b ON a.dt = b.dt
ORDER BY a.dt;
"""

In [13]:
sql_request(query)

,dt,roi
0,2016-03-01,383.18
1,2016-04-01,362.01
2,2016-05-01,352.05
3,2016-06-01,323.89
4,2016-07-01,313.46
...,...,...
59,2021-02-01,330.74
60,2021-03-01,340.33
61,2021-04-01,327.70
62,2021-05-01,339.32


In [14]:
engine.dispose()